## 1️⃣ Cài đặt thư viện

In [ ]:
%%capture
# Cài đặt PyTorch và PyTorch Geometric
!pip install torch==2.1.0 torchvision==0.16.0 --index-url https://download.pytorch.org/whl/cu118
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv -f https://data.pyg.org/whl/torch-2.1.0+cu118.html
!pip install torch-geometric==2.4.0
!pip install scikit-learn pandas numpy pyyaml tqdm matplotlib seaborn

In [ ]:
# Kiểm tra GPU
import torch
print(f"PyTorch: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## 2️⃣ Kiểm tra Dataset

In [ ]:
import os
import json
import numpy as np
import pandas as pd
from pathlib import Path

# Đường dẫn dataset - THAY ĐỔI theo tên dataset bạn upload
DATASET_PATH = '/kaggle/input/cicids2018-processed'

print("📁 Files trong dataset:")
if os.path.exists(DATASET_PATH):
    files = sorted(os.listdir(DATASET_PATH))
    for f in files:
        fpath = os.path.join(DATASET_PATH, f)
        if os.path.isfile(fpath):
            size_mb = os.path.getsize(fpath) / (1024**2)
            print(f"  ✓ {f:<60} ({size_mb:.1f} MB)")
else:
    print(f"  ❌ Không tìm thấy: {DATASET_PATH}")
    print("  👉 Hãy upload dataset và sửa DATASET_PATH")

In [ ]:
# Load manifest
manifest_path = os.path.join(DATASET_PATH, 'manifest.json')
if os.path.exists(manifest_path):
    with open(manifest_path, 'r') as f:
        manifest = json.load(f)
    
    print("\n📊 Thông tin dataset:")
    print(f"  Mode: {manifest['mode']}")
    print(f"  Total samples: {manifest['total_samples']:,}")
    print(f"  Features: {manifest['num_features']}")
    print(f"  Train: {manifest['splits']['train']['size']:,}")
    print(f"  Val: {manifest['splits']['val']['size']:,}")
    print(f"  Test: {manifest['splits']['test']['size']:,}")

## 3️⃣ Load Data & Extract IP Information

Cần đọc CSV files để lấy thông tin Src IP và Dst IP cho mỗi flow.

In [ ]:
print("Loading preprocessed data...")
X = np.load(os.path.join(DATASET_PATH, 'X.npy'))
y = np.load(os.path.join(DATASET_PATH, 'y.npy'))
idx_train = np.load(os.path.join(DATASET_PATH, 'idx_train.npy'))
idx_val = np.load(os.path.join(DATASET_PATH, 'idx_val.npy'))
idx_test = np.load(os.path.join(DATASET_PATH, 'idx_test.npy'))

print(f"\n✓ Loaded:")
print(f"  X: {X.shape}")
print(f"  y: {y.shape}")
print(f"  Train: {len(idx_train):,}, Val: {len(idx_val):,}, Test: {len(idx_test):,}")

## ⚠️ Sampling Strategy (Optional but Recommended)

**Dataset rất lớn (11M+ flows)!** Build graph và training sẽ mất nhiều thời gian.

**Khuyến nghị:**
- Để thử nghiệm nhanh: `SAMPLE_SIZE = 500_000` (500K flows, ~10-15 phút)
- Để kết quả tốt hơn: `SAMPLE_SIZE = 2_000_000` (2M flows, ~45-90 phút)
- Để train full: `SAMPLE_SIZE = None` (11M+ flows, nhiều giờ!)

In [ ]:
# 🎯 SAMPLING CONFIG - THAY ĐỔI THEO NHU CẦU
SAMPLE_SIZE = 1_000_000  # None = full dataset, 1M = 1 triệu samples
RANDOM_SEED = 42

if SAMPLE_SIZE is not None and SAMPLE_SIZE < len(X):
    print(f"\n⚡ Sampling {SAMPLE_SIZE:,} flows from {len(X):,} total...")
    
    # Stratified sampling to keep class balance
    from sklearn.model_selection import train_test_split
    
    # Sample indices while preserving class distribution
    all_indices = np.arange(len(X))
    sampled_indices, _ = train_test_split(
        all_indices, 
        train_size=SAMPLE_SIZE,
        stratify=y,
        random_state=RANDOM_SEED
    )
    sampled_indices.sort()  # Keep order
    
    # Create mapping from old to new indices
    old_to_new = {old_idx: new_idx for new_idx, old_idx in enumerate(sampled_indices)}
    
    # Sample data
    X = X[sampled_indices]
    y = y[sampled_indices]
    
    # Remap train/val/test indices
    idx_train_new = [old_to_new[i] for i in idx_train if i in old_to_new]
    idx_val_new = [old_to_new[i] for i in idx_val if i in old_to_new]
    idx_test_new = [old_to_new[i] for i in idx_test if i in old_to_new]
    
    idx_train = np.array(idx_train_new)
    idx_val = np.array(idx_val_new)
    idx_test = np.array(idx_test_new)
    
    print(f"\n✓ Sampled:")
    print(f"  X: {X.shape}")
    print(f"  Train: {len(idx_train):,}, Val: {len(idx_val):,}, Test: {len(idx_test):,}")
    
    # Check class distribution
    unique, counts = np.unique(y, return_counts=True)
    print(f"\n📊 Sampled class distribution:")
    for cls, cnt in zip(unique, counts):
        print(f"  Class {cls}: {cnt:,} ({cnt/len(y)*100:.1f}%)")
else:
    print(f"\n✓ Using FULL dataset: {len(X):,} flows")

In [ ]:
# Load CSV files để lấy IP addresses
print("\nLoading CSV files to extract IP addresses...")

csv_files = [f for f in os.listdir(DATASET_PATH) if f.endswith('_cleaned.csv')]
csv_files.sort()

print(f"Found {len(csv_files)} CSV files")

# Load all CSV files
dfs = []
for csv_file in csv_files:
    print(f"  Reading {csv_file}...")
    df = pd.read_csv(os.path.join(DATASET_PATH, csv_file))
    dfs.append(df)

# Concatenate all
df_all = pd.concat(dfs, ignore_index=True)
print(f"\n✓ Total flows in CSV: {len(df_all):,}")

# Verify số lượng khớp với X, y
if len(df_all) != len(X):
    print(f"⚠️ WARNING: CSV rows ({len(df_all)}) != X rows ({len(X)})")
    print("   Đảm bảo CSV files khớp với X.npy, y.npy")
else:
    print(f"✓ Number of flows matches!")

# Extract IP columns
src_ips = df_all['Src IP'].values
dst_ips = df_all['Dst IP'].values

print(f"\n✓ Extracted IP addresses:")
print(f"  Unique Source IPs: {len(set(src_ips)):,}")
print(f"  Unique Destination IPs: {len(set(dst_ips)):,}")
print(f"  Total unique IPs: {len(set(src_ips) | set(dst_ips)):,}")

# Free memory
del dfs, df_all
import gc
gc.collect()

## 4️⃣ Build IP Graph

Tạo graph structure:
- **Nodes**: Mỗi unique IP là một node
- **Edges**: Mỗi flow tạo một edge từ Src IP → Dst IP
- **Edge labels**: Label của flow (benign/attack)
- **Node features**: Aggregate features từ các flows liên quan

In [ ]:
from collections import defaultdict
from tqdm.auto import tqdm
import time

print("🌐 Building IP-based graph...")
print(f"   Dataset size: {len(X):,} flows")
start_time = time.time()

# Step 1: Create IP to node ID mapping
print(f"\n[1/6] 🗺️  Mapping IPs to node IDs...")
step_start = time.time()

with tqdm(total=3, desc="Creating mappings", bar_format='{l_bar}{bar}| {elapsed}') as pbar:
    all_ips = set(src_ips) | set(dst_ips)
    pbar.update(1)
    
    ip_to_id = {ip: idx for idx, ip in enumerate(sorted(all_ips))}
    pbar.update(1)
    
    id_to_ip = {idx: ip for ip, idx in ip_to_id.items()}
    pbar.update(1)

num_nodes = len(ip_to_id)
print(f"      ✓ Created {num_nodes:,} nodes ({time.time()-step_start:.1f}s)")

# Step 2: Create edge list from flows
print(f"\n[2/6] 🔗 Creating edge list from flows...")
step_start = time.time()

edge_src = []
edge_dst = []
edge_labels = []

for i in tqdm(range(len(src_ips)), desc="Processing flows", 
              bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'):
    src_id = ip_to_id[src_ips[i]]
    dst_id = ip_to_id[dst_ips[i]]
    
    edge_src.append(src_id)
    edge_dst.append(dst_id)
    edge_labels.append(y[i])

edge_index = torch.tensor([edge_src, edge_dst], dtype=torch.long)
edge_labels = torch.tensor(edge_labels, dtype=torch.long)

print(f"      ✓ Created {edge_index.shape[1]:,} edges ({time.time()-step_start:.1f}s)")

# Step 3: Aggregate node features (from training flows only)
print(f"\n[3/6] 📊 Aggregating node features (from training data only)...")
print(f"      Using {len(idx_train):,} training flows to prevent data leakage")
step_start = time.time()

# Initialize feature aggregation
node_feature_sum = defaultdict(lambda: np.zeros(X.shape[1]))
node_feature_count = defaultdict(int)

# Only use training flows
for i in tqdm(idx_train, desc="Aggregating features", 
              bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'):
    src_id = ip_to_id[src_ips[i]]
    dst_id = ip_to_id[dst_ips[i]]
    
    # Aggregate features for both source and destination IPs
    node_feature_sum[src_id] += X[i]
    node_feature_count[src_id] += 1
    
    node_feature_sum[dst_id] += X[i]
    node_feature_count[dst_id] += 1

print(f"      Computing mean features...")
node_features = np.zeros((num_nodes, X.shape[1]), dtype=np.float32)
for node_id in tqdm(range(num_nodes), desc="Computing means",
                    bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt}'):
    if node_feature_count[node_id] > 0:
        node_features[node_id] = node_feature_sum[node_id] / node_feature_count[node_id]

print(f"      ✓ Node features: {node_features.shape} ({time.time()-step_start:.1f}s)")

# Step 4: Create node labels (majority vote)
print(f"\n[4/6] 🏷️  Creating node labels (majority vote)...")
step_start = time.time()

node_label_votes = defaultdict(lambda: [0, 0])  # [benign_count, attack_count]

for i in tqdm(range(len(src_ips)), desc="Voting labels",
              bar_format='{l_bar}{bar}| {n_fmt}/{total_fmt} [{elapsed}<{remaining}]'):
    src_id = ip_to_id[src_ips[i]]
    dst_id = ip_to_id[dst_ips[i]]
    
    label = y[i]
    node_label_votes[src_id][label] += 1
    node_label_votes[dst_id][label] += 1

# Majority vote
node_labels = np.zeros(num_nodes, dtype=np.int64)
for node_id in range(num_nodes):
    votes = node_label_votes[node_id]
    node_labels[node_id] = 1 if votes[1] > votes[0] else 0

benign_count = np.sum(node_labels==0)
attack_count = np.sum(node_labels==1)
print(f"      ✓ Node labels: benign={benign_count:,}, attack={attack_count:,} ({time.time()-step_start:.1f}s)")

# Free memory
print(f"\n[5/6] 🧹 Cleaning up memory...")
del node_feature_sum, node_feature_count, node_label_votes
gc.collect()
print(f"      ✓ Memory freed")

In [ ]:
# Step 6: Create edge splits based on flow splits
print(f"\n[6/6] 🎭 Creating edge train/val/test splits...")
step_start = time.time()

with tqdm(total=3, desc="Creating masks", bar_format='{l_bar}{bar}| {elapsed}') as pbar:
    train_edge_mask = np.zeros(len(edge_labels), dtype=bool)
    train_edge_mask[idx_train] = True
    pbar.update(1)
    
    val_edge_mask = np.zeros(len(edge_labels), dtype=bool)
    val_edge_mask[idx_val] = True
    pbar.update(1)
    
    test_edge_mask = np.zeros(len(edge_labels), dtype=bool)
    test_edge_mask[idx_test] = True
    pbar.update(1)

train_edges = edge_index[:, train_edge_mask]
val_edges = edge_index[:, val_edge_mask]
test_edges = edge_index[:, test_edge_mask]

print(f"      ✓ Train edges: {train_edges.shape[1]:,}")
print(f"      ✓ Val edges: {val_edges.shape[1]:,}")
print(f"      ✓ Test edges: {test_edges.shape[1]:,}")
print(f"      ({time.time()-step_start:.1f}s)")

# Create PyG Data
print(f"\n📊 Creating PyG Data object...")
from torch_geometric.data import Data

data = Data(
    x=torch.tensor(node_features, dtype=torch.float32),
    edge_index=edge_index,
    edge_y=edge_labels,  # Edge labels
    y=torch.tensor(node_labels, dtype=torch.long)  # Node labels (for reference)
)

total_time = time.time() - start_time
print(f"\n{'='*60}")
print(f"✅ GRAPH BUILDING COMPLETE!")
print(f"{'='*60}")
print(f"⏱️  Total time: {total_time:.1f}s ({total_time/60:.1f} min)")
print(f"📊 Graph statistics:")
print(f"   • Nodes (IPs): {data.num_nodes:,}")
print(f"   • Edges (flows): {data.edge_index.shape[1]:,}")
print(f"   • Features per node: {data.x.shape[1]}")
print(f"   • Node labels: {benign_count:,} benign, {attack_count:,} attack")
print(f"   • Train edges: {train_edges.shape[1]:,}")
print(f"   • Val edges: {val_edges.shape[1]:,}")
print(f"   • Test edges: {test_edges.shape[1]:,}")
print(f"{'='*60}\n")

## 5️⃣ Define Model

Edge classification model - predict label for each edge (flow).

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv

class IPGraphSAGE(nn.Module):
    """GraphSAGE for IP-based edge classification."""
    
    def __init__(self, in_dim, hidden_dim, num_classes, num_layers=2, dropout=0.2, aggr='mean'):
        super().__init__()
        self.num_layers = num_layers
        self.dropout = dropout
        
        # Node embedding layers
        self.convs = nn.ModuleList()
        self.convs.append(SAGEConv(in_dim, hidden_dim, aggr=aggr))
        
        for _ in range(num_layers - 1):
            self.convs.append(SAGEConv(hidden_dim, hidden_dim, aggr=aggr))
        
        # Edge classifier
        self.edge_mlp = nn.Sequential(
            nn.Linear(hidden_dim * 2, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )
    
    def forward(self, x, edge_index, edge_label_index=None):
        # Node embeddings
        for i, conv in enumerate(self.convs):
            x = conv(x, edge_index)
            if i < len(self.convs) - 1:
                x = F.relu(x)
                x = F.dropout(x, p=self.dropout, training=self.training)
        
        # If edge_label_index provided, classify those edges
        if edge_label_index is not None:
            src_emb = x[edge_label_index[0]]
            dst_emb = x[edge_label_index[1]]
            edge_emb = torch.cat([src_emb, dst_emb], dim=1)
            return self.edge_mlp(edge_emb)
        
        return x

# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

model = IPGraphSAGE(
    in_dim=X.shape[1],
    hidden_dim=128,
    num_classes=2,
    num_layers=2,
    dropout=0.2,
    aggr='mean'
).to(device)

print(f"\n✓ Model created:")
print(model)
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")

## 6️⃣ Training

Train model to classify edges (flows) as benign or attack.

In [ ]:
# Move data to device
data = data.to(device)

# Config
EPOCHS = 50  # Giảm từ 100 -> 50 cho dataset lớn
BATCH_SIZE = 4096  # Tăng batch size (IP-GNN có ít edges hơn Flow-GNN)
LR = 0.001
WEIGHT_DECAY = 0.0001
PATIENCE = 10  # Early stopping

# Optimizer and loss
optimizer = torch.optim.Adam(model.parameters(), lr=LR, weight_decay=WEIGHT_DECAY)
criterion = nn.CrossEntropyLoss()

print(f"✓ Setup complete:")
print(f"  Epochs: {EPOCHS}")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {LR}")

In [ ]:
def evaluate_edges(model, data, edge_indices):
    """Evaluate model on given edges."""
    model.eval()
    with torch.no_grad():
        out = model(data.x, data.edge_index, edge_indices)
        pred = out.argmax(dim=1).cpu().numpy()
        true = data.edge_y[edge_indices[0]].cpu().numpy()  # Edge labels
        
        acc = accuracy_score(true, pred)
        f1 = f1_score(true, pred, average='weighted', zero_division=0)
        precision = precision_score(true, pred, average='weighted', zero_division=0)
        recall = recall_score(true, pred, average='weighted', zero_division=0)
        
        return {
            'accuracy': acc,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }

# Training loop
best_val_f1 = 0.0
patience_counter = 0
history = {'train_loss': [], 'val_acc': [], 'val_f1': []}

print("\n🚀 Starting training...\n")

for epoch in range(1, EPOCHS + 1):
    # Train
    model.train()
    total_loss = 0
    num_batches = 0
    
    # Mini-batch training over edges
    num_train_edges = train_edges.shape[1]
    indices = torch.randperm(num_train_edges)
    
    for start_idx in range(0, num_train_edges, BATCH_SIZE):
        end_idx = min(start_idx + BATCH_SIZE, num_train_edges)
        batch_indices = indices[start_idx:end_idx]
        
        batch_edges = train_edges[:, batch_indices]
        batch_labels = data.edge_y[train_edge_mask][batch_indices]
        
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, batch_edges)
        loss = criterion(out, batch_labels)
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    train_loss = total_loss / num_batches
    
    # Evaluate on validation edges
    val_metrics = evaluate_edges(model, data, val_edges)
    
    history['train_loss'].append(train_loss)
    history['val_acc'].append(val_metrics['accuracy'])
    history['val_f1'].append(val_metrics['f1'])
    
    # Print progress
    print(f"Epoch {epoch:3d}/{EPOCHS} | "
          f"Loss: {train_loss:.4f} | "
          f"Val Acc: {val_metrics['accuracy']:.4f} | "
          f"Val F1: {val_metrics['f1']:.4f}")
    
    # Save best model
    if val_metrics['f1'] > best_val_f1:
        best_val_f1 = val_metrics['f1']
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_f1': best_val_f1
        }, 'best_ip_gnn_model.pt')
        patience_counter = 0
    else:
        patience_counter += 1
    
    # Early stopping
    if patience_counter >= PATIENCE:
        print(f"\n⚠️ Early stopping at epoch {epoch}")
        break

print(f"\n✓ Training complete! Best Val F1: {best_val_f1:.4f}")

## 7️⃣ Final Evaluation

In [ ]:
# Load best model
checkpoint = torch.load('best_ip_gnn_model.pt')
model.load_state_dict(checkpoint['model_state_dict'])
print(f"✓ Loaded best model from epoch {checkpoint['epoch']}")

# Evaluate on test edges
test_metrics = evaluate_edges(model, data, test_edges)

print("\n" + "="*60)
print("📊 FINAL TEST RESULTS")
print("="*60)
print(f"Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"F1 Score:  {test_metrics['f1']:.4f}")
print(f"Precision: {test_metrics['precision']:.4f}")
print(f"Recall:    {test_metrics['recall']:.4f}")
print("="*60)

In [ ]:
# Detailed classification report
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

model.eval()
with torch.no_grad():
    out = model(data.x, data.edge_index, test_edges)
    y_pred = out.argmax(dim=1).cpu().numpy()
    y_true = data.edge_y[test_edge_mask].cpu().numpy()

print("\n📋 Classification Report:")
print(classification_report(y_true, y_pred, target_names=['Benign', 'Attack']))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Benign', 'Attack'],
            yticklabels=['Benign', 'Attack'])
plt.title('Confusion Matrix - IP GNN')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.tight_layout()
plt.savefig('ip_gnn_confusion_matrix.png', dpi=150, bbox_inches='tight')
plt.show()

## 8️⃣ Training History Plot

In [ ]:
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Loss
axes[0].plot(history['train_loss'], label='Train Loss', linewidth=2)
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].set_title('Training Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Metrics
axes[1].plot(history['val_acc'], label='Val Accuracy', linewidth=2)
axes[1].plot(history['val_f1'], label='Val F1', linewidth=2)
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Score')
axes[1].set_title('Validation Metrics')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ip_gnn_training_history.png', dpi=150, bbox_inches='tight')
plt.show()

## 9️⃣ Analyze IP-level Statistics

In [ ]:
# Analyze some IP statistics
print("📊 IP-level Statistics:\n")

# Top IPs by degree
from collections import Counter

ip_degrees = Counter()
for i in range(edge_index.shape[1]):
    src = edge_index[0, i].item()
    dst = edge_index[1, i].item()
    ip_degrees[src] += 1
    ip_degrees[dst] += 1

print("Top 10 IPs by degree (number of connections):")
for node_id, degree in ip_degrees.most_common(10):
    ip = id_to_ip[node_id]
    label = "Attack" if node_labels[node_id] == 1 else "Benign"
    print(f"  {ip:<20} degree: {degree:>6,}  label: {label}")

## 📥 Download Model

Bạn có thể download model đã train về máy từ Kaggle Output.

In [ ]:
# Model đã được lưu tại: best_ip_gnn_model.pt
print("✓ Model saved at: best_ip_gnn_model.pt")
print("✓ Confusion matrix: ip_gnn_confusion_matrix.png")
print("✓ Training history: ip_gnn_training_history.png")